# Inference Sample for ProtT5nv

SPDX-FileCopyrightText: Copyright (c) <year> NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: LicenseRef-NvidiaProprietary

NVIDIA CORPORATION, its affiliates and licensors retain all intellectual property and proprietary rights in and to this material, related documentation and any modifications thereto. Any use, reproduction, disclosure or distribution of this material and related documentation without an express license agreement from NVIDIA CORPORATION or its affiliates is strictly prohibited.

### Prerequisite

**Before diving in, ensure you have all [necessary prerequisites](https://docs.nvidia.com/bionemo-framework/latest/pre-reqs.html). If this is your first time using BioNeMo, we recommend following the [quickstart guide](https://docs.nvidia.com/bionemo-framework/latest/quickstart-fw.html) first.** 

Additionally, this notebook assumes you have started a [local inference server](https://docs.nvidia.com/bionemo-framework/latest/inference-triton-fw.html) using a pretrained [Prot-T5](https://docs.nvidia.com/bionemo-framework/latest/models/prott5nv.html) model.


In [ ]:
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from pathlib import Path
import os

try:
    BIONEMO_HOME: Path = Path(os.environ['BIONEMO_HOME']).absolute()
except KeyError:
    print("Must have BIONEMO_HOME set in the environment! See docs for instructions.")
    raise

config_path = BIONEMO_HOME / "examples" / "protein" / "prott5nv" / "conf"
print(f"Using model configuration at: {config_path}")
assert config_path.is_dir()

### Setup and Test Data

In [ ]:
seqs = [
    'MSLKRKNIALIPAAGIGVRFGADKPKQYVEIGSKTVLEHVL', 
    'MIQSQINRNIRLDLADAILLSKAKKDLSFAEIADGTGLA',
]

In [ ]:
from bionemo.triton.utils import load_model_config

cfg = load_model_config(config_path, config_name="infer.yaml")

In [ ]:
# from bionemo.model.loading import setup_inference
# inferer, trainer, dataloader = setup_inference(cfg, interactive=True)
# NEEDS the data for the trainer & dataloader
# i.e. needs /workspace/bionemo/data/FLIP
from bionemo.triton.utils import load_model_for_inference
from bionemo.model.protein.prott5nv.infer import ProtT5nvInference

inferer = load_model_for_inference(cfg, interactive=True)

print(f"Loaded a {type(inferer)}")
assert isinstance(inferer, ProtT5nvInference)

### Sequence to Hidden States

__`seq_to_hiddens`__ queries the model to fetch the encoder hiddens states for the input protein sequence. `pad_mask` is returned with `hidden_states` and contains padding information  

In [ ]:
hidden_states, pad_masks = inferer.seq_to_hiddens(seqs)
print(f"{hidden_states.shape=}")
print(f"{pad_masks.shape=}")
assert tuple(hidden_states.shape) == (2, 41, 768)
assert tuple(pad_masks.shape) == (2, 41)

### Hidden States to Embedding

__`hiddens_to_embedding`__ computes embedding vector by averaging `hidden_states` 

In [ ]:
embeddings = inferer.hiddens_to_embedding(hidden_states, pad_masks)
print(f"{embeddings.shape=}")
assert tuple(embeddings.shape) == (2, 768)

### Sequence to Embedding

__`seq_to_embedding`__  queries the model to fetch the encoder hiddens states and computes embedding vector

In [ ]:
embeddings = inferer.seq_to_embeddings(seqs)
print(f"{embeddings.shape=}")
assert tuple(embeddings.shape) == (2, 768)